In [1]:
import csv 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
import time

ModuleNotFoundError: No module named 'bs4'

In [1]:
arx_book = openpyxl.load_workbook(r'C:\Users\Andy\Documents\3A\PROJET ARX\Projet Srapping_API - Partie 2.xlsx')
book = openpyxl.load_workbook(r'C:\Users\Andy\Documents\3A\PROJET ARX\resultats.xlsx')
writer= pd.ExcelWriter(r'C:\Users\Andy\Documents\3A\PROJET ARX\resultats.xlsx',engine='openpyxl')
writer.book = book

NameError: name 'openpyxl' is not defined

In [9]:
sheet = arx_book.active

rows = sheet.max_row 
#la fonction ne renvoie pas la dernière ligne non-vide mais toutes les lignes d'où la condition !=None en bas
cols = sheet.max_column

SIRET = []
noms = []
pays = []

for row in sheet.iter_rows(min_row=2, min_col=2, max_row=rows, max_col=2) :
    for cell in row:
        if cell.value != None :
            noms.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=3, max_row=rows, max_col=3) :
    for cell in row:
        if cell.value != None : 
            SIRET.append(cell.value)
        
for row in sheet.iter_rows(min_row=2, min_col=4, max_row=rows, max_col=4) :
    for cell in row:
        if cell.value != None : 
            pays.append(cell.value)
    
data = pd.DataFrame({'SIRET': SIRET, 'pays': pays, },
                      index = noms)

data

,SIRET,pays
AG2R La Mondiale,502 858 418 00028,France
Astera,570 501 551 00016,France
Groupe PRO BTP,394 164 966 00019,France
Crédit Mutuel,588 505 354 00013,France
Système U,304 602 956 00142,France
...,...,...
Greenergy,2809935,Royaume-Uni
Ineos,3851680,Royaume-Uni
Wittington Investments,366054,Royaume-Uni
Betfred Group,7717019,Royaume-Uni


In [10]:
SIRET_test = ['50285841800028','41759197100011']
noms_test = ['AG2R La Mondiale']
prefixe = 'https://www.manageo.fr/entreprises/'
suffixe = '.html'

def get_pages(U):
    pages = []
    for i in range(0,len(U)):
        SIREN = U[i]
        j = prefixe + SIREN[:-5] + suffixe #enlever les 5 derniers pour avoir le SIREN à partir du SIRET
        pages.append(j)
    return pages

pages_test = get_pages(SIRET_test)
pages_test

['https://www.manageo.fr/entreprises/502858418.html',
 'https://www.manageo.fr/entreprises/417591971.html']

In [11]:
import re
   
pages = []

for i in range(0,len(data)):
    if data['pays'][i] == 'France':
        SIRET = data['SIRET'][i]
        SIRET = re.sub(' ',  '',SIRET)
        SIRET = re.sub('\u200e',  '',SIRET) #une des lignes avait ce truc
        j = prefixe + SIRET[:-5] + suffixe #enlever les 5 derniers pour avoir le SIREN à partir du SIRET
        pages.append(j)        
        
pages

['https://www.manageo.fr/entreprises/502858418.html',
 'https://www.manageo.fr/entreprises/570501551.html',
 'https://www.manageo.fr/entreprises/394164966.html',
 'https://www.manageo.fr/entreprises/588505354.html',
 'https://www.manageo.fr/entreprises/304602956.html',
 'https://www.manageo.fr/entreprises/383986874.html',
 'https://www.manageo.fr/entreprises/450527916.html',
 'https://www.manageo.fr/entreprises/775577018.html',
 'https://www.manageo.fr/entreprises/421343369.html',
 'https://www.manageo.fr/entreprises/407794551.html',
 'https://www.manageo.fr/entreprises/642880785.html',
 'https://www.manageo.fr/entreprises/417591971.html',
 'https://www.manageo.fr/entreprises/493455042.html',
 'https://www.manageo.fr/entreprises/882245293.html',
 'https://www.manageo.fr/entreprises/752610147.html',
 'https://www.manageo.fr/entreprises/356000000.html',
 'https://www.manageo.fr/entreprises/642007991.html',
 'https://www.manageo.fr/entreprises/343973822.html',
 'https://www.manageo.fr/ent

In [44]:
def count_chr(char,texte):
    count = 0
    for i in texte: 
        if i == char: 
            count = count + 1
    return count

In [18]:
list_adresse = []

for k in range(0,len(pages_test)):
    r = requests.get(pages_test[k]).text #récupèrer les adresses html de la liste "pages"
    soup = BeautifulSoup(r,'html.parser') #permet de récupérer le code html de la page
    alldata1 = soup.find_all("div",{"id":"datas-etablissement","class":"col-md-10 col-xs-12"}) #la partie html concernant les adresses
    for a in range(len(alldata1)): 
        data = alldata1[a].find_all("div") #les lignes par ligne de cette partie html
        adresse = ""
        for b in range(len(data)): #récupération du texte avec l'adresse
            bout_de_texte = data[b].text
#             bout_de_texte=re.sub("^\s+|\s+$","",bout_de_texte)
            adresse = adresse + bout_de_texte
            adresse = re.sub('voir la carte',  '',adresse) #enlève les textes parasytes
            adresse = re.sub('\n',  '',adresse)
            adresse = re.sub('\r',  '',adresse)
    list_adresse.append(adresse)


print(list_adresse)


['14-1614 BD MALESHERBES,                                                                                                  75008 PARIS 8', '38 RUE FRANCOIS PEISSEL,                                                                                                  69300 CALUIRE-ET-CUIRE']


In [19]:
tableau_info = []

for k in range(0,len(pages_test)):
    r = requests.get(pages_test[k]).text #récupèrer les adresses html de la liste "pages"
    soup = BeautifulSoup(r,'html.parser') #permet de récupérer le code html de la page
    alldata = soup.find_all("td",{"class":"col-xs-7"}) #la partie html concernant les informations
    for a in range(len(alldata)): 
        data = alldata[a].find_all("td") #les lignes par ligne de cette partie html
        information = ""
        for b in range(len(data)): #récupération du texte
            bout_de_texte = data[b].text
            information = information + bout_de_texte
    tableau_info.append(information)

print(tableau_info)


['', '']


In [123]:
alldata = soup.find_all("div",{"id":"card-infos-juridiques","class":"card-content"})
for a in range(len(alldata)): 
    data = alldata[a].find_all("div")
    for b in range(len(data)):
        str = data[b].text
        print(str)

FR32502858418




Code greffe
7501
                  : Paris


N° dossier
2008D01217


Raison sociale RCS
SGAM AG2R LA MONDIALE


Date immatriculation RCS
13/03/2008


Date de création siège actuel
30/12/2019








              D'INFOS
            







Code greffe
7501
                  : Paris


N° dossier
2008D01217


Raison sociale RCS
SGAM AG2R LA MONDIALE


Date immatriculation RCS
13/03/2008


Date de création siège actuel
30/12/2019








              D'INFOS
            







              D'INFOS
            





In [ ]:
for k in range(len(data)):
    r = requests.get(pages[k]).text #permet d'accéder à la page 
#     time.sleep(5)
    soup = BeautifulSoup(r,'html.parser') #permet de récupérer le code html de la page
    alldata = soup.find_all("div",{"class":"D(tbr) fi-row Bgc($hoverBgColor):h"}) #alldata correspond aux balises de toutes les données qu'on veut récupérer
    alldata1 = soup.find_all("div",{"class":"D(tbr) C($primaryColor)"})#alldata1 correspond aux balises qui contiennent les dates (années) des données qu'on récupère
    #Cette boucle récupère les années des données à partir de alldata1
    for a in range(len(alldata1)): 
        year =[]
        data = alldata1[a].find_all("div")
        for b in range(len(data)):
            year.append(data[b].text) 
    tableau = {} #On va stocker les données de l'entreprise dans un dictionnaire pour le transformer ensuite en dataframe
    #Cette boucle récupère les données à partir de alldata et les place dans le dictionnaire 
    for i in range(len(alldata)):
        data = alldata[i].find_all("div")
        l = []
        for j in range(3,len(data)):
            l.append(data[j].text.replace('\u202f',' '))
        tableau[data[0].text]=l 
    df = pd.DataFrame.from_dict(tableau,orient='index') #On transforme les données récupérées sous forme de dataframe, ce qui permet de les visualiser sous la forme d'un tableau
    df.columns = year[2:] #On écrit les années qui correspondent aux colonnes du tableau
    #df.to_excel(r'C:\Users\lorke\Projet_DDEFI\.spyproject\''+noms[k]+'.xlsx') #à utiliser si on veut un fichier excel par entreprise
    df.to_excel(writer,sheet_name=noms[k]) #à utiliser si on veut les résultats dans un même fichier Excel, avec une page par entreprise